In [ ]:
%pip install langchain-community python-dotenv "unstructured[md]" ragas pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
import os
from dotenv import load_dotenv
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.testset import TestsetGenerator


In [ ]:

path = "datasets/"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [ ]:

load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable")



In [ ]:

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

## Generate Testset

Now we will run the test generation using the loaded documents and the LLM setup. If you have used llama_index to load documents, please use generate_with_llama_index_docs method instead.

In [ ]:
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlineSplitter:   0%|          | 0/12 [00:00<?, ?it/s]         unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
Applying SummaryExtractor:  67%|██████▋   | 4/6 [00:05<00:01,  1.01it/s]Property 'summary' already exists in node '910559'. Skipping!
Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/30 [00:00<?, ?it/s]Property 'summary_embedding' already exists in node '910559'. Skipping!
Generating Samples: 100%|██████████| 12/12 [00:09<00:00,  1.29it/s]


In [ ]:
dataset_df = dataset.to_pandas()

In [ ]:
dataset_df.head()

,user_input,reference_contexts,reference,synthesizer_name
0,"Wht iz the role of Mentoring at GitLab, and ho...","[title: ""Sales Sponsorship Pilot Program"" desc...",Mentoring at GitLab is defined as an opportuni...,single_hop_specifc_query_synthesizer
1,What is the role of a sponsor in the GitLab Sp...,[Sponsorship Program As a Sponsor you should b...,A sponsor in the GitLab Sponsorship Program us...,single_hop_specifc_query_synthesizer
2,Wht iz the role of a sponsee at GitLab?,[Role of a Sponsee A Sponsee at GitLab will be...,"A Sponsee at GitLab, particularly in the pilot...",single_hop_specifc_query_synthesizer
3,Cud yu pleese explane wut DIB meens in the con...,"[title: ""Building an Inclusive Remote Culture""...","DIB stands for Diversity, Inclusion & Belongin...",single_hop_specifc_query_synthesizer
4,How does self-education contribute to being an...,"[<1-hop>\n\ntitle: ""The Ally Lab"" description:...",Self-education is the first step in being an a...,multi_hop_abstract_query_synthesizer


In [ ]:
dataset_df.to_csv("testset.csv", index=False)

### Dataset diversity

The dataset must be able to cover the use cases of our application:
It must have the following in it:
* Curriculem books
* Wikipedia articles
* Youtube videos